In [1]:
import MDAnalysis as mda
import numpy as np

/home/chilpert/miniconda3/envs/graphene/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chilpert/miniconda3/envs/graphene/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:160: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:

u = mda.Universe('input.gro')
c = np.unique(u.atoms.positions[:,1]) # finding the unique y-coordinate so that we can loop through each row of atoms
u.atoms.masses = 36    # Since it's a TC5 bead, we use a mass of 36 for each

/home/chilpert/miniconda3/envs/graphene/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: X
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/chilpert/miniconda3/envs/graphene/lib/python3.11/site-packages/MDAnalysis/coordinates/GRO.py:228: UserWarning: Empty box [0., 0., 0.] found - treating as missing unit cell. Dimensions set to `None`.
  warnings.warn(wmsg)


In [3]:
for j in range(len(c)): # Looping through each row, defined by the y-coordinate
    if (j !=0) and (j !=len(c)-1):
        if j % 2 !=0:
            group = u.atoms[u.atoms.positions[:,1] == c[j]]
            gr = np.arange(1, len(group), 3)
         
            for k in gr:
                u.atoms[group[k].index].mass = 0
            
        else:
            group = u.atoms[u.atoms.positions[:,1] == c[j]]
            gr = np.arange(2, len(group), 3)
            for k in gr:
                u.atoms[group[k].index].mass = 0
                

In [4]:
v = u.atoms[u.atoms.masses !=0]

In [5]:
def hexagon(universe):
    
    ''' Identifying the vertics of hexagon around a virtual site '''
    
    b = u.atoms[u.atoms.masses == 0].indices
    hexagon_indices = []
    for i in b:
        empty = []
        for j in u.atoms.indices:
            if (2.55 <= calc_bonds(u.atoms[j].position, u.atoms[i].position) <= 2.57):
                empty.append(j)
        hexagon_indices.append(empty)
    return hexagon_indices

In [6]:
from MDAnalysis.lib.distances import calc_angles, calc_bonds, calc_dihedrals

In [7]:
hexagons = hexagon(u)

In [8]:
def get_angles(hexagons):
    angles = []
    for hex in hexagons:
        angle1 = [hex[2], hex[4], hex[0], hex[5]]
        angle2 = [hex[4], hex[0], hex[5], hex[1]]
        angle3 = [hex[0], hex[5], hex[1], hex[3]]
        angles.append(angle1)
        angles.append(angle2)
        angles.append(angle3)
    return angles

In [26]:
angles = get_angles(hexagons)

In [27]:
angles

[[4, 10, 0, 11],
 [10, 0, 11, 1],
 [0, 11, 1, 6],
 [7, 13, 2, 14],
 [13, 2, 14, 3],
 [2, 14, 3, 9],
 [11, 17, 6, 18],
 [17, 6, 18, 7],
 [6, 18, 7, 13],
 [15, 21, 10, 22],
 [21, 10, 22, 11],
 [10, 22, 11, 17],
 [18, 23, 13, 24],
 [23, 13, 24, 14],
 [13, 24, 14, 20]]

In [23]:
def get_between_hexagons_angles(hexagons, beads_per_row = 6):
    def get_common_side(hexa1, hexa2):
        common = list(set(hexa1).intersection(set(hexa2)))
        common.sort()
        return common
    
    angles = []
    gap_for_above = int(beads_per_row / 2)
    for i in range(len(hexagons)):
        hexa = hexagons[i]
        try:
            hexa_after = hexagons[i+2]
        except IndexError:
            hexa_after = None
        try: 
            hexa_above = hexagons[i+gap_for_above]
        except IndexError:
            hexa_above = None
        
        try: 
            hexa_before = hexagons[i - 1]
        except IndexError:
            hexa_before = None
        
        if hexa_after:
            side_after = get_common_side(hexa, hexa_after)
            if side_after:
                idx1 = hexa.index(side_after[0])
                idx2 = hexa_after.index(side_after[1])
                angle = [hexa[idx1 - 2], side_after[0], side_after[1], hexa_after[idx2 + 2]]
                angles.append(angle)
            
        
        if hexa_above: 
            side_above = get_common_side(hexa, hexa_above)
            if side_above: 
                idx1 = hexa.index(side_above[0])
                idx2 = hexa_above.index(side_above[1])
                angle = [hexa[idx1 - 4], side_above[0], side_above[1], hexa_above[idx2 + 4]]
                angles.append(angle)  
                
        if hexa_before:
            side_before = get_common_side(hexa, hexa_before)
            if side_before:
                idx1 = hexa_before.index(side_before[0])
                idx2 = hexa.index(side_before[1])
                angle = [hexa_before[idx1 - 2], side_before[0], side_before[1], hexa[idx2 + 2]]
                angles.append(angle)
    return angles

In [24]:
other_angles = get_between_hexagons_angles(hexagons)

In [25]:
other_angles

[[1, 6, 11, 17],
 [0, 10, 11, 22],
 [2, 13, 14, 24],
 [7, 13, 18, 23],
 [2, 7, 13, 18],
 [6, 11, 17, 22]]